# Explanation on generator design choices. 

**Don't look if you don't want spoilers!**



# Scope of project (Notespace --> probably will go in README)

There are several parts to the project.

1. Write generator, generate data
2. Explore generated data, decide what to test
3. Write test plan for experiment
4. Regenerate data
5. Explore data, interpret findings
6. Present/document results

I've included three notebooks explaining choices behind each component

1. Generator (generating and regenerating)
2. Exploration + selecting test target
3. Test case + experiment findings

To play with the data... [give download, requirements.txt instructions].

To run the entire project...[same instructions as above]
* then run the generator - you can change the months and years values to generate a different amount of data (python is fine)
* you should have a bunch of .csv files now. do some exploratory analysis
* figure out what you want to test, then regenerate the data - optional to change weights/values - it should print out .txt files correlating to your new weights
* explore the new data, determine if it's a statistically significant change. then you can look at the .txt file to verify.

##### (!) I need more practice on what's actually statistically significant!

##### Generator summary

This notebook covers the design choices of the data generator (and changes for the regenerator). If you're looking for an exploratory analysis of the data, look [HERE] If you're looking for the experimental plan and analysis, look [HERE].

# Generator features

### Popular times (by hour)

Used Google popular times to estimate orders per hour. Assigned weights and assumed 0 value gives 50%.

Huge variance - busiest day is almost 3x as busy as least busy day.

---

I noticed Google has "popular_times" values for the restaurant. There isn't really a scale, but I can use these to approximate restaurant occupancy to the hour instead of to the day.

I'm not sure how specific these are to 2020 - they seem a little low to me, but I figured it's fine because I'm just estimating and I'll throw in variance anyway.

##### (!) should I add in variance in the years? and the months?! how did i miss that?

##### (!) add in a visualization that shows the min/max values vs. the average weighted values we have? that would be a cool graph! (would belong in this notebook).

### Market sizing

Seating count, dining time, delivery/takeout

### Annual/seasonal multipliers

Year multiplier, summer/winter month multiplier. Mostly negative.

### Chance for additional item in order

If ordering starter or dessert, there's a chance for an additional item. 

### Chance for increased order quantity

### Generating by hour

Tried by month (because csvs are by month), but "accuracy" to the hour made more sense. Added variance with normal distribution.

### Weights for menu (by category, by item)

Categories have weights, and then within the category the items have weights.

# Data REgenerator

We regenerate fresh data to simulate collection of user data. 

Current plan is slight random variance in weights, and printing out the weight used to a .txt file which I won't look at until after the analysis. The goal is to evaluate whether or not the value went up or down, and whether it's statistically significant or just noise (e.g. the change we made actually had some impact). 

I'll look at the randomly printed value afterwards to compare. Might make a note: based on the test results, I would say this is statistically significant and would recommend this change going forward. Bonus: since it's a simulation, I'll try to guess the weight (not really sure if this is worth anything since there's so many random variation all over the place, but it's just for fun).